# Modeling
## Ames Housing Data 

This is the second of two notebooks for the Ames Housing data project. Here, I look at various linear regression models and examine feature combinations that minimize root mean squared error (RMSE) and Mean Absolute Error (MAE), as well as increase R2 scores, when it comes to predicting `SalesPrice`.


This notebook contains 4 models. For each model, I've consolidated the code and print statements into two cells in order make the code more readable. Beneath each model is an interpretation of the scores.

Note that the models progressively increase in both complexity and how well they perform. This is deliberate and intended to show the development and refinement of my approach. 

# Contents
- [Import Libraries and Datasets](#Import-Libraries-and-Datasets)
- [Model 0: Baseline Score](#Baseline-Score)
- [Model 1: Increasing Number of Features](#Increasing-Number-of-Features)
- [Model 2: Introduce Log of y](#Introduce-Log-of-y)
- [Model 3: Stretch Model](#Stretch-Model)
- [Conclusions](#Conclusions)

# Import Libraries and Datasets

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno

from sklearn.linear_model import LinearRegression, Lasso, LassoCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from math import sqrt

from scipy import stats
import statsmodels.api as sm

%matplotlib inline

train = pd.read_csv('./datasets/train_processed.csv')
test = pd.read_csv('./datasets/test_processed.csv')

### First, let's get reacquainted with the data and revisit the problem statement.

In [2]:
# view shapes
print('train shape:',(train.shape))
print('test shape:',(test.shape))

train shape: (2049, 304)
test shape: (878, 286)


In [3]:
# view feature correlations
pd.set_option('display.max_rows', 305)
train.corr().SalePrice.sort_values(ascending=False)

SalePrice               1.000000
Overall Qual            0.803462
Gr Liv Area             0.719463
Total Bsmt SF           0.665116
Garage Area             0.655097
1st Flr SF              0.648252
Garage Cars             0.648227
Bsmt Qual_Ex            0.589810
Year Built              0.572405
Kitchen Qual_Ex         0.555131
Year Remod/Add          0.550872
Full Bath               0.538225
Foundation_PConc        0.529500
Mas Vnr Area            0.511273
TotRms AbvGrd           0.509775
Exter Qual_Ex           0.500425
Fireplaces              0.473783
BsmtFin Type 1_GLQ      0.464266
Heating QC_Ex           0.453582
Neighborhood_NridgHt    0.448639
Exter Qual_Gd           0.446721
BsmtFin SF 1            0.446103
Garage Finish_Fin       0.423776
Fireplace Qu_Gd         0.385490
Bsmt Exposure_Gd        0.379081
Sale Type_New           0.360599
Garage Type_Attchd      0.358104
Exterior 1st_VinylSd    0.342156
Open Porch SF           0.338545
Lot Frontage            0.338280
Exterior 2

### Problem Statement:
*Create a regression model based on the Ames Housing Dataset that predicts the price of a house at sale.*

With that in mind, we'll start to model.

# Baseline score
**Model 0:** First, I'll calculate a baseline score on a basic model using the top 2 features positively correlated to `SalePrice`. 

In [4]:
# create feature matrix (X) and target vector(y)
X0 = train[['Gr Liv Area', 'Overall Qual']]
y = train['SalePrice']

# train/test split
X0_train, X0_test, y0_train, y0_test = train_test_split(X0, y, random_state=42) 

# instantiate and fit model
lr0 = LinearRegression()
lr0.fit(X0_train, y0_train)

# calculate predictions and MSE 
preds0 = lr0.predict(X0_test)
resids0 = y0_test - preds0
mse0 = mean_squared_error(y0_test, preds0)

# print regression metrics
print('RMSE', np.sqrt(mse0))
print('')
print('MAE', (np.mean(np.abs(resids0))))
print('')
print('cross val r2', cross_val_score(lr0, X0_train, y0_train, cv=5).mean())
print('train score r2', lr0.score(X0_train, y0_train))
print('test r2', lr0.score(X0_test, y0_test))
print('')
print('coef',lr0.coef_)
print('inter',lr0.intercept_)

RMSE 39513.14132862921

MAE 28585.646485026253

cross val r2 0.7421838494662962
train score r2 0.7477165023413908
test r2 0.7530433334607132

coef [   63.74405567 32104.71386231]
inter -109592.99904293506


### Interpretation:
- *I will include a more detailed interpretation of metrics here. Moving forward, I will assess the regression metrics relative to metrics from other models.*
- RMSE tells us that the square root of our average squared error is 39,513 USD. Considering the mean sale price of a home in the train data set 181,480 USD, this isn't a RMSE to get too excited about.
    - *NOTE: I will not be interpreting coefficients and intercepts moving forward due to the volume of features in following models.*
- That said, our model is well-fitted considering the R2 scores are all relatively close to one another. Typically, an R2 score of ~0.745 is decent, however, when we consider the RMSE relative to sale price, we know there's room for improvement.
- Our MAE tells us that, on average, our predictions are off by 28,585 USD. Again, considering the mean price of a home is 181,480 USD, this is not an MAE I'd feel comfortable settling with.
- Finally, our coefficients tell us that, all else held equal, we can expect home prices to increase 64 USD and 32,104 USD for every unit increase, respectively, in `Gr Liv Area` and `Overall Qual`
    - `Gr Liv Area` is above ground living area square feet.
    - `Overall Qual` rates the overall material and finish of the house. It is measured on a scale from 1-10 with 1 being very poor and 10 being very excellent.

- We're off to a good start but our model can definitely use plenty of fine tuning.

# Increasing Number of Features
**Model 1:** The features included in this model are a result of trial and error feature combination. I took various combinations of the top 15 positively and negatively correlated features to `SalePrice` and took the best RMSE. 

In [5]:
# instanstiate, fit
X1 = train[['Overall Qual', 
             'Garage Area', 
             '1st Flr SF', 
             'MS SubClass', 
             'Lot Frontage', 
             'Year Built',
             'Year Remod/Add',
             'Full Bath',
             'TotRms AbvGrd',
             'Fireplaces',
             'Heating QC_Ex',
             'Neighborhood_NridgHt',
             'Exter Qual_TA',
             'Open Porch SF',
             'Wood Deck SF',
             'Central Air_Y'
            ]]

y = train[['SalePrice']]
           
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y, random_state=42)  
               
lr1 = LinearRegression()
lr1.fit(X_train1, y_train1)

preds1 = lr1.predict(X_test1)
resids1 = y_test1 - preds1
mse1 = mean_squared_error(y_test1, preds1)

print('RMSE', np.sqrt(mse1))
print('')
print('MAE', (np.mean(np.abs(resids1))))
print('')
print('cross val r2', cross_val_score(lr1, X_train1, y_train1, cv=5).mean())
print('train score r2', lr1.score(X_train1, y_train1))
print('test r2', lr1.score(X_test1, y_test1))

RMSE 32300.18475429842

MAE SalePrice    23784.448143
dtype: float64

cross val r2 0.8121741733881803
train score r2 0.8200235125782036
test r2 0.8349757941007054


### Interpretation:
- Here, we see an increase in scores across the board.
- One thing that's particularly interesting to note is that negative correlations, such as `Exter Qual_TA`, which is an exterior rating of typical/average, help to increase our RMSE and R2 scores.
- Similar to last model, we can tell that our model is well-fit (i.e., a good balance between bias and variance) due to the relative consistency between our R2 scores.
- Using these same features, let's try taking the log of y in our next model.

# Introduce Log of y
**Model 2:** In this model, I'm interested in see how the log of `SalePrice` will affect our regression metrics. I will keep the features consistent with Model 1, but will re-fit our model with the log of `SalePrice`. Since I am keeping the features consistent with Model 1, I will use the same variables when taking logs.

In [6]:
# take log of y
y_train1_log = np.log(y_train1)
y_test1_log = np.log(y_test1)

In [7]:
# fit with log
lr1.fit(X_train1, y_train1_log)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [8]:
preds = lr1.predict(X_test1) # logarithms
resids_log = y_test1_log - preds
resids_exp =  np.exp(y_test1_log) - np.exp(preds)
mse1_log = mean_squared_error(y_test1, np.exp(preds))

In [9]:
print('RMSE', np.sqrt(mse1_log))
print('')
print('MAE', (np.mean(np.abs(resids_exp))))
print('')
print(f'cross val r2 score: {cross_val_score(lr1, X_train1, y_train1_log, cv=5).mean()}')
print(f'train log r2 score: {lr1.score(X_train1, y_train1_log)}')
print(f'test log r2 score: {lr1.score(X_test1, y_test1_log)}')

RMSE 26801.819856406357

MAE SalePrice    19521.342209
dtype: float64

cross val r2 score: 0.8508212473920175
train log r2 score: 0.8566111838871895
test log r2 score: 0.8403108089449278


### Interpretation:
- By taking the log of y we see a clear improvement from model 1 in all of our metrics. RMSE decreased by ~5,500 and MAE decreased by ~5,300. 
- Moreover, each of the R2 scores increased slightly while maintaining relative consistency. This suggests that the model continues to be well fit.
- *NOTE: this is the model that I submitted to the Kaggle competition.*

# Stretch Model
**Model 3:** I call this my stretch model because I wasn't able to submit it to the Kaggle competition due to time constraints. I came up with it at the 11th hour and did not have bandwidth to figure out how to adjust my test variables so as to fit the training model. Still, I wanted to include the model here should I come back to revisit this work.

In [10]:
# create feature matrix (X) and target vector(y)
X3 = train._get_numeric_data().drop(columns=['SalePrice'])
features = list(X3.columns)
y = train['SalePrice']

# train/test split
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y, random_state=42)

# instantiate and fit model
lr3 = LinearRegression()
lr3.fit(X3_train, y3_train)

# calculate predictions and MSE 
preds3 = lr3.predict(X3_test)
resids3 = y3_test - preds3
mse3 = mean_squared_error(y3_test, preds3)

In [11]:
# print regression metrics
print('RMSE', np.sqrt(mse3))
print('')
print('MAE', (np.mean(np.abs(resids3))))
print('')
print('cross val r2', cross_val_score(lr3, X3_train, y3_train, cv=5).mean())
print('train score r2', lr3.score(X3_train, y3_train))
print('test r2', lr3.score(X3_test, y3_test))

RMSE 20823.39529228111

MAE 15109.988257625359

cross val r2 0.8999863711370437
train score r2 0.9428608043240398
test r2 0.9314131708210244


### Interpretation and elaboration:
- As is clear, this is by far and away the best model yet. The RMSE is ~6,000 less than that of model 2 and MAE ~4,400 less. What's R2 scores are also better than those scores in other models.
- Consistent with the other models, this stretch model appears well-fit when considering the relative similarity in R2 scores. The improved R2 scores suggest that this model is even better fit.
- This model is vastly different than the other models given the number of features it includes (303). Many of these features are dummies created on categorical data in the original training set.
- Given more time, I'd like to take a more hypothesis-driven approach to tuning the current features and also to possibly create interaction terms.

# Conclusions

- At a high level, there are two clear themes when it comes to fitting a good MLR model, based on my models. First, taking the log of y, SalePrice, increases the model’s performance by all measures used. We saw this between Model 1 and Model 2. Second, increasing the number of dependent variables is generally a good thing. We observed this between Model 0 to Model 1 as well as Model 2 to Model 3. In the case of the latter, the increase was substantial (from 16 features to 303 features). Certainly, this leaves plenty of room for further investigation and fine tuning.
- The features included in Models 2&3 illustrate that it can be important to include both positively and negatively correlated variables. This can be counterintuitive, in that people generally associate positively correlated features to predicting a target y. However, when we get our coefficients on well-chosen negatively correlated features, this can help to decrease residuals.
- Given more time, I would have liked to tune Model 3. Techniques would include overfitting then scaling back, further feature engineering such as interaction terms, as well as a more hypothesis-driven approach to curating variables.